# Baseline - Linear Interpolation

For a ticker, given a sample period (e.g. 3-1-2009 to 12-31-2015), we design a linear interpolation method to to predict its closing price on 3-31-2019:  

predicted_price = price_start + (price_end - price_start) * (end_label - start)/(end - start)  
10_bagger = predicted_price > price_start*10  
 
where:  
* price_start - first valid price in the sample period
* price_end - last valid price in the sample period
* end_label - 3-31-2019
* start - date of the first valid price in the sample period
* end - date of the last valid price in the sample period

This serves as the baseline for other more advanced machine learning methods (e.g. logistic regression) for 10 Bagger prediction.


In [1]:
import quandl  # Access to Sharadar Core US Equities Bundle
api_key = '7B87ndLPJbCDzpNHosH3'

import math
import platform
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
import torch
import pandas as pd
from IPython.display import display
import time

from datetime import date, datetime, time, timedelta


print("Python version: ", platform.python_version())
print("Pytorch version: {}".format(torch.__version__))

Python version:  3.6.6
Pytorch version: 1.1.0


## Import Labels

For each sample period (e.g. 3-1-2009 to 12-31-2018), we want to import a list of valid tickers. A valid ticker is defined as a ticker which is active for at least 180 days before the end of the sample period. 

For example, if the end of the sample period is 12-31-2018, a ticker has to be active since 7-4-2018. Any ticker that IPO after 7-4-2018 is not a valid ticker, since there is no enough price history to make an educated prediction.

In [100]:
labels = pd.read_csv("../datasets/sharader/labels_12-31-2018.csv")

y = labels.set_index('ticker')
y['firstpricedate']= pd.to_datetime(y['firstpricedate'])
y['lastpricedate']= pd.to_datetime(y['lastpricedate'])

y.head()

,appreciation,10bagger,table,permaticker,name,exchange,isdelisted,category,cusips,siccode,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
ticker,,,,,,,,,,,,,,,,,,,,,
A,6.339117,False,SEP,196290,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
AA,1.224348,False,SEP,124392,Alcoa Corp,NYSE,N,Domestic,013872106,3350.0,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
AAAGY,1.275556,False,SEP,120538,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAAP,3.331837,False,SEP,155760,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,2834.0,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAC,0.099459,False,SEP,187592,AAC Holdings Inc,NYSE,Y,Domestic,000307108,8093.0,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


### Number of active tickers

In [101]:
tickers = list(y.index)
print(len(tickers))

9881


In [87]:
valid_tickers = pd.Series(tickers, name = 'ticker')

valid_tickers.head()

0        A
1    AAAGY
2     AACC
3     AACG
4      AAI
Name: ticker, dtype: object

In [88]:
prices = pd.read_csv("../datasets/sharader/inputs_notfilled_2010-12-31.csv")
prices

,date,A,AAAGY,AACC,AACG,AAI,AAIIQ,AAIR,AAL,AAME,...,ZRAN,ZRBA,ZSTN,ZUMZ,ZURVY,ZVO,ZVUE,ZYTO,ZYXI,ZZ
0,2009-03-02,12.68,15.75,3.29,5.18,2.54,1.61,NaN,2.47,0.637,...,5.66,1.55,NaN,7.45,12.75,NaN,0.01,0.011,1.21,0.84
1,2009-03-03,12.68,15.75,3.30,5.32,2.46,1.61,NaN,2.30,0.610,...,5.70,1.40,NaN,7.19,12.85,NaN,0.01,0.011,1.22,0.76
2,2009-03-04,13.31,16.35,3.33,5.08,2.78,1.61,NaN,2.37,0.660,...,6.22,1.40,NaN,7.17,13.74,NaN,0.01,0.011,1.22,0.76
3,2009-03-05,12.54,15.59,3.30,5.08,2.56,1.53,NaN,1.97,0.600,...,6.16,1.40,NaN,6.69,11.91,NaN,0.01,0.011,1.17,0.58
4,2009-03-06,12.65,15.97,3.40,5.25,2.89,1.80,NaN,2.16,0.510,...,6.17,1.22,NaN,6.43,11.30,NaN,0.01,0.011,1.20,0.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,2010-12-27,41.92,NaN,6.04,3.71,7.39,1.50,2.26,9.87,2.000,...,8.42,NaN,6.66,28.83,25.50,18.90,NaN,0.110,0.55,2.94
461,2010-12-28,41.51,NaN,5.96,3.71,7.40,1.70,2.40,9.89,2.000,...,8.57,NaN,6.60,28.65,25.80,18.82,NaN,0.110,0.55,2.95
462,2010-12-29,41.69,NaN,6.02,3.69,7.38,1.50,2.40,10.14,2.000,...,8.74,NaN,6.57,28.21,25.88,19.14,NaN,0.110,0.58,2.98
463,2010-12-30,41.59,NaN,5.92,3.65,7.39,1.49,2.32,10.13,2.050,...,8.65,NaN,6.61,27.71,25.93,18.94,NaN,0.110,0.62,2.92


In [89]:
X = prices.set_index('date')
print (X['A'])

date
2009-03-02    12.68
2009-03-03    12.68
2009-03-04    13.31
2009-03-05    12.54
2009-03-06    12.65
              ...  
2010-12-27    41.92
2010-12-28    41.51
2010-12-29    41.69
2010-12-30    41.59
2010-12-31    41.43
Name: A, Length: 465, dtype: float64


In [22]:
start_date_sample = datetime.strptime('2009-03-02', '%Y-%m-%d')
end_date_sample = datetime.strptime('2018-12-31', '%Y-%m-%d')
end_date_label = datetime.strptime('2019-3-31', '%Y-%m-%d')

(end_date_label-start_date_sample).days 

3681

## Linear Interpolate

In [90]:
# Start and end date of the sampled period
start_date_sample = datetime.strptime('2009-03-02', '%Y-%m-%d')
end_date_sample = datetime.strptime('2010-12-31', '%Y-%m-%d')
end_date_label = datetime.strptime('2019-3-31', '%Y-%m-%d')

# List for saving predicted price on 3-31-2019
predicts = []

for ticker in valid_tickers:
    
    # print("Ticker: {}".format(ticker))
    
    # First and last dates when the ticker is active (datetime)
    first_price_date = y['firstpricedate'].loc[ticker]
    last_price_date = y['lastpricedate'].loc[ticker]
    
    # print("First Trading Date: {}".format(first_price_date.strftime('%Y-%m-%d')))
    # print("Last Trading Date: {}".format(last_price_date.strftime('%Y-%m-%d')))
    
    # If the ticker IPO after the start of the sample period, 
    if first_price_date > start_date_sample:  
        start = first_price_date  # set start to the date of the first valid price
    else:
        start = start_date_sample  # otherwise set start to the beginning of the sample period
        
    price_start = X.loc[start.strftime('%Y-%m-%d'), ticker]

    # print("Start: {}  Price: {}".format(start,price_start))

    # If the ticker is delisted ahead of the end of the sample period, 
    if last_price_date < end_date_sample: 
        # The price on the date of the delisting is the final price, no need to predict
        # using linear interpolation
        predict = X.loc[last_price_date.strftime('%Y-%m-%d'), ticker]

    else:
        end = end_date_sample  # otherwise set end to the end of the sample period
        price_end = X.loc[end.strftime('%Y-%m-%d'), ticker]

        # print("Last: {}  Price: {}".format(end, price_end))
        
        predict = price_start + (price_end-price_start)*(end_date_label-start).days/(end-start).days

    appreciation = predict/price_start
    Is_10_Bagger = appreciation > 10.0
    # print("Predicted Price: {}  Appreciation: {}    10 Bagger: {}".format(predict, appreciation, Is_10_Bagger))

    predicts.append([ticker,appreciation,Is_10_Bagger])


In [91]:
df = pd.DataFrame(predicts, columns =['ticker', 'appreciation','10bagger'])

df.to_csv("../datasets/sharader/predicts_lin_int_12-31-2010.csv")

In [102]:
df = pd.read_csv("../datasets/sharader/predicts_lin_int_12-31-2018.csv")

In [103]:
df['10bagger'].value_counts()

False    9406
True      475
Name: 10bagger, dtype: int64

In [114]:
def calc_metrics(predictions, labels):
    # Calculate True positives, false positives, etc.

    TP_ = np.logical_and(predictions, labels)
    FP_ = np.logical_and(predictions, np.logical_not(labels))
    TN_ = np.logical_and(np.logical_not(predictions), np.logical_not(labels))
    FN_ = np.logical_and(np.logical_not(predictions), labels)

    TP=sum(TP_)
    FP=sum(FP_)
    TN=sum(TN_)
    FN=sum(FN_)
    
    return TP,FP,TN,FN

def calc_error_rates(TP, FP, TN, FN):
    # Calculate precision, recall, accuracy, TPR, TNR and BER
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    BER = 1.0 - (TPR+TNR)/2
    return precision, recall, accuracy, TPR, TNR, BER

In [115]:
TP,FP,TN,FN = calc_metrics(list(df['10bagger']),list(y['10bagger']))

print(TP,FP,TN,FN)

precision, recall, accuracy, TPR, TNR, BER = calc_error_rates(TP, FP, TN, FN)

print(precision, recall, accuracy, TPR, TNR, BER)

465 10 9340 66
0.9789473684210527 0.8757062146892656 0.9923084708025504 0.8757062146892656 0.9989304812834224 0.06268165201365594


In [118]:
end_dates = ['2010-12-31',
             '2011-12-31',
             '2012-12-31',
             '2013-12-31',
             '2014-12-31',
             '2015-12-31',
             '2016-12-31',
             '2017-12-31',
             '2018-12-31'
            ]

label_filenames = ['labels_12-31-2010.csv',
              'labels_12-31-2011.csv',
              'labels_12-31-2012.csv',
              'labels_12-31-2013.csv',
              'labels_12-31-2014.csv',
              'labels_12-31-2015.csv',
              'labels_12-31-2016.csv',
              'labels_12-31-2017.csv',
              'labels_12-31-2018.csv'
             ]

predict_filenames = ['predicts_lin_int_12-31-2010.csv',
              'predicts_lin_int_12-31-2011.csv',
              'predicts_lin_int_12-31-2012.csv',
              'predicts_lin_int_12-31-2013.csv',
              'predicts_lin_int_12-31-2014.csv',
              'predicts_lin_int_12-31-2015.csv',
              'predicts_lin_int_12-31-2016.csv',
              'predicts_lin_int_12-31-2017.csv',
              'predicts_lin_int_12-31-2018.csv'
             ]

for date, label_filename, predict_filename in zip(end_dates, label_filenames, predict_filenames):
    
    print(date)
    
    labels = pd.read_csv("../datasets/sharader/"+label_filename)
    predicts = pd.read_csv("../datasets/sharader/"+predict_filename)
    
    TP,FP,TN,FN = calc_metrics(list(predicts['10bagger']),list(labels['10bagger']))

    print(TP,FP,TN,FN)

    precision, recall, accuracy, TPR, TNR, BER = calc_error_rates(TP, FP, TN, FN)

    print("Precision:{:.3f} Recall:{:.3f}".format(precision, recall))
    print("Accuracy:{:.3f} TPR:{:.3f} TNR:{:.3f} BER:{:.3f}".format(accuracy, TPR, TNR, BER))


2010-12-31
387 1405 4633 125
Precision:0.216 Recall:0.756
Accuracy:0.766 TPR:0.756 TNR:0.767 BER:0.238
2011-12-31
319 588 5757 196
Precision:0.352 Recall:0.619
Accuracy:0.886 TPR:0.619 TNR:0.907 BER:0.237
2012-12-31
332 437 6158 188
Precision:0.432 Recall:0.638
Accuracy:0.912 TPR:0.638 TNR:0.934 BER:0.214
2013-12-31
399 560 6338 125
Precision:0.416 Recall:0.761
Accuracy:0.908 TPR:0.761 TNR:0.919 BER:0.160
2014-12-31
415 402 6920 112
Precision:0.508 Recall:0.787
Accuracy:0.935 TPR:0.787 TNR:0.945 BER:0.134
2015-12-31
395 181 7992 135
Precision:0.686 Recall:0.745
Accuracy:0.964 TPR:0.745 TNR:0.978 BER:0.138
2016-12-31
418 156 8331 113
Precision:0.728 Recall:0.787
Accuracy:0.970 TPR:0.787 TNR:0.982 BER:0.116
2017-12-31
463 155 8718 68
Precision:0.749 Recall:0.872
Accuracy:0.976 TPR:0.872 TNR:0.983 BER:0.073
2018-12-31
465 10 9340 66
Precision:0.979 Recall:0.876
Accuracy:0.992 TPR:0.876 TNR:0.999 BER:0.063


In [125]:
(1-0.216)**10

0.08773252460082347